In [1]:
import os
import tqdm

import torch

In [2]:
ckpt_path_1 = "llama-2-70b/consolidated.0"
ckpt_path_2 = ".pth"

ckpt_path_lst = [ckpt_path_1+str(i)+ckpt_path_2 for i in range(8)]

In [3]:
ckpt_lst = []
for ckpt_path in tqdm.tqdm(ckpt_path_lst):
    ckpt_lst.append(torch.load(ckpt_path))

100%|██████████| 8/8 [01:09<00:00,  8.75s/it]


In [12]:
ckpt_lst[0].keys()

dict_keys(['tok_embeddings.weight', 'norm.weight', 'output.weight', 'layers.0.attention.wq.weight', 'layers.0.attention.wk.weight', 'layers.0.attention.wv.weight', 'layers.0.attention.wo.weight', 'layers.0.feed_forward.w1.weight', 'layers.0.feed_forward.w2.weight', 'layers.0.feed_forward.w3.weight', 'layers.0.attention_norm.weight', 'layers.0.ffn_norm.weight', 'layers.1.attention.wq.weight', 'layers.1.attention.wk.weight', 'layers.1.attention.wv.weight', 'layers.1.attention.wo.weight', 'layers.1.feed_forward.w1.weight', 'layers.1.feed_forward.w2.weight', 'layers.1.feed_forward.w3.weight', 'layers.1.attention_norm.weight', 'layers.1.ffn_norm.weight', 'layers.2.attention.wq.weight', 'layers.2.attention.wk.weight', 'layers.2.attention.wv.weight', 'layers.2.attention.wo.weight', 'layers.2.feed_forward.w1.weight', 'layers.2.feed_forward.w2.weight', 'layers.2.feed_forward.w3.weight', 'layers.2.attention_norm.weight', 'layers.2.ffn_norm.weight', 'layers.3.attention.wq.weight', 'layers.3.atten

In [14]:
for i, ckpt in enumerate(ckpt_lst):
    print(i, ckpt['norm.weight'].shape)

0 torch.Size([8192])
1 torch.Size([8192])
2 torch.Size([8192])
3 torch.Size([8192])
4 torch.Size([8192])
5 torch.Size([8192])
6 torch.Size([8192])
7 torch.Size([8192])


In [5]:
for ckpt_ in ckpt_lst:
    print(ckpt_['layers.9.attention.wk.weight'].shape)

torch.Size([128, 8192])
torch.Size([128, 8192])
torch.Size([128, 8192])
torch.Size([128, 8192])
torch.Size([128, 8192])
torch.Size([128, 8192])
torch.Size([128, 8192])
torch.Size([128, 8192])


In [22]:
ckpt_lst[0]["layers.1.attention_norm.weight"].dtype

torch.bfloat16

In [4]:
from llama import Llama, Tokenizer

In [5]:
tokenizer_path = "tokenizer.model"
tokenizer = Tokenizer(tokenizer_path)

In [8]:
tokenizer.encode("asdfasdfads", False, False)

[408, 2176, 294, 2176, 7925]

In [37]:
keys_lst = []
for i, ckpt in enumerate(ckpt_lst):
    keys_lst.append(list(ckpt.keys()))
print(len(keys_lst))

for i, keys in enumerate(keys_lst):
    for j, key_ in enumerate(keys):
        for k, keys_ in enumerate(keys_lst):
            if key_ not in keys_:
                print(i, j, k, key_)

8


In [38]:
def check_colpar(str):
    column_parallel_name_lst = ["output", "wq", "wk", "wv", "w1", "w3"]
    for colpar_name in column_parallel_name_lst:
        if colpar_name in str:
            return True
    return False

def check_rowpar(str):
    row_parallel_name_lst = ["embeddings", "wo", "w2"]
    for rowpar_name in row_parallel_name_lst:
        if rowpar_name in str:
            return True
    return False

def check_nopar(str):
    no_parallel_name_lst = ["rope", "norm"]
    for nopar_name in no_parallel_name_lst:
        if nopar_name in str:
            return True
    return False

In [48]:
model_weights_dict = {}
for i, model_weights_name in enumerate(tqdm.tqdm(keys_lst[0])):
    
    a = None
    for j in range(7):
        
        if a is None:
            if check_colpar(model_weights_name):
                a = torch.concat((ckpt_lst[j][model_weights_name],
                                  ckpt_lst[j+1][model_weights_name]), dim=0)
            elif check_rowpar(model_weights_name):
                a = torch.concat((ckpt_lst[j][model_weights_name],
                                  ckpt_lst[j+1][model_weights_name]), dim=1)
            elif check_nopar(model_weights_name):
                a = ckpt_lst[j][model_weights_name]
                break
            else:
                assert True, "not matching"
        else:
            if check_colpar(model_weights_name):
                a = torch.concat((a,
                                  ckpt_lst[j+1][model_weights_name]), dim=0)
            elif check_rowpar(model_weights_name):
                a = torch.concat((a,
                                  ckpt_lst[j+1][model_weights_name]), dim=1)
            else:
                assert True, "not matching"
    
    model_weights_dict[model_weights_name] = a

len(model_weights_dict)

100%|██████████| 724/724 [01:28<00:00,  8.19it/s]


724

In [50]:
# for model_name in keys_lst[0]:
#     print(model_name, model_weights_dict[model_name].shape)

In [51]:
torch.save(model_weights_dict,"./whole_llama2_model.pt")